In [101]:
import spacy
import pandas as pd
import numpy as np
import csv
import random

In [100]:
import nltk
# nltk.download('all')

### Building a corpus from the given 'data.txt' file
#### Reduced the corpus to first 1500 lines for faster execution

In [6]:
from google.colab import files

def read_file(filename):
  uploaded = files.upload()

  a_file = open(filename, "r")

  str_final = ""

  for line in a_file:
    str_stripped = line.rstrip()
    str_final += ' ' + str_stripped
  
  a_file.close()

  return str_final

In [7]:
corpus=read_file('data_new.txt')

Saving data_new.txt to data_new.txt


In [8]:
corpus

' sai kha ya her kisi kay bus ki bat nhi hai lakin main ki hal kal bi aj aur aj bi sirf aus say bus kya bt hai wah je wah are wha kaya bat hai wah kya baat likhi ya allah rehm farma hm sab pe or zalimo ko hidayat de ameen allah insaaf karey je thik kaha right jee ye to he wah wah yeh toh hy jee ye to he wha wha kaya bat hai bohat khoob so true alhamdulillah right mara jasa masom log mashaallah mubarak ma hun hero tara feel kar speed bot ki great aadmi aadmi he has one son with her mein inho ne pehli martaba british ka tour kiya to wahan bhi dhoom macha di is tarah wo aaj tak sab ke lala he hain she received an award of pride of performance and women life achievement award from benazir bhutto sanjeeda acting comedy parody singing hosting aur ab drama nigari jis se in ki do betiyan hain bushra apni family se bohat ziyada attached hain aur fursat ka tamam waqt ghar walon ke sath he guzarti hain is ke ilawa wo show ke deegar roles ke liye bhi songs gaya karti thin comedy shows ki bat ho to

In [9]:
urdu=spacy.blank('ur')

roman_text=urdu(corpus)

### Creating unigrams out of the corpus

In [10]:
def n_grams(token, n):
  return list((token[i:i+n] for i in range(len(token)-n+1)))

In [11]:
unigrams=n_grams(roman_text, 1)

In [12]:
unigrams

[ ,
 sai,
 kha,
 ya,
 her,
 kisi,
 kay,
 bus,
 ki,
 bat,
 nhi,
 hai,
 lakin,
 main,
 ki,
 hal,
 kal,
 bi,
 aj,
 aur,
 aj,
 bi,
 sirf,
 aus,
 say,
 bus,
 kya,
 bt,
 hai,
 wah,
 je,
 wah,
 are,
 wha,
 kaya,
 bat,
 hai,
 wah,
 kya,
 baat,
 likhi,
 ya,
 allah,
 rehm,
 farma,
 hm,
 sab,
 pe,
 or,
 zalimo,
 ko,
 hidayat,
 de,
 ameen,
 allah,
 insaaf,
 karey,
 je,
 thik,
 kaha,
 right,
 jee,
 ye,
 to,
 he,
 wah,
 wah,
 yeh,
 toh,
 hy,
 jee,
 ye,
 to,
 he,
 wha,
 wha,
 kaya,
 bat,
 hai,
 bohat,
 khoob,
 so,
 true,
 alhamdulillah,
 right,
 mara,
 jasa,
 masom,
 log,
 mashaallah,
 mubarak,
 ma,
 hun,
 hero,
 tara,
 feel,
 kar,
 speed,
 bot,
 ki,
 great,
 aadmi,
 aadmi,
 he,
 has,
 one,
 son,
 with,
 her,
 mein,
 inho,
 ne,
 pehli,
 martaba,
 british,
 ka,
 tour,
 kiya,
 to,
 wahan,
 bhi,
 dhoom,
 macha,
 di,
 is,
 tarah,
 wo,
 aaj,
 tak,
 sab,
 ke,
 lala,
 he,
 hain,
 she,
 received,
 an,
 award,
 of,
 pride,
 of,
 performance,
 and,
 women,
 life,
 achievement,
 award,
 from,
 benazir,
 bhutto,

### For generating error tables

In [13]:
insert_table = []
delete_table = []
substitute_table = []
transpose_table = []

rows = cols = 26

In [14]:
for i in range(rows):
    col = []
    
    for j in range(cols):
        col.append(0)
    
    delete_table.append(col)
    insert_table.append(col)
    substitute_table.append(col)
    transpose_table.append(col)

# delete_table

In [15]:
def get_alphabet_index(char): 
  return ord(char) - 96

In [16]:
get_alphabet_index('a')

1

In [18]:
from Levenshtein import distance, editops

### Test function to demonstrate the insert, delete, transpose, and substitute values and indexes

In [40]:
def test(src, dest): 
  src = src.lower()
  dest = dest.lower()

  editop_list = editops(src, dest) #edit distance

  return editop_list

print('hwllo --> hello')
print(test('hello', 'hwllo'))
print()

print('hllo --> hello')
print(test('hello', 'hllo'))
print()

print('hlelo --> hello')
print(test('hello', 'hlelo'))
print()

print('helloo --> hello')
print(test('hello', 'helloo'))
print()

hwllo --> hello
[('replace', 1, 1)]

hllo --> hello
[('delete', 1, 1)]

hlelo --> hello
[('replace', 1, 1), ('replace', 2, 2)]

helloo --> hello
[('insert', 5, 5)]



### Populating the Error tables using the edit distance from source word to destination word given the above 4 (insert, delete etc.) scenarios

In [37]:
def populate_mappings_lists(src, dest): 
  src = src.lower()
  dest = dest.lower()

  editop_list = editops(src, dest) #edit distance

  for op in range(len(editop_list)): 
    
    if (editop_list[op][0] == 'delete'): 
      x = get_alphabet_index(src[editop_list[op][1]-1]) - 1
      y = get_alphabet_index(src[editop_list[op][2]]) - 1
      #print('delete', x, y)
      delete_table[x][y] = delete_table[x][y] + 1

    elif (editop_list[op][0] == 'insert'): 
      x = get_alphabet_index(src[editop_list[op][1]-1]) - 1
      y = get_alphabet_index(dest[editop_list[op][2]]) - 1
      #print('insert', x, y)
      insert_table[x][y] = insert_table[x][y] + 1

    # transpose
    elif (op < len(editop_list)-2 and editop_list[op][0] == 'replace' and editop_list[op+1][0] == 'replace'): 
      x = get_alphabet_index(src[editop_list[op][1]]) - 1
      y = get_alphabet_index(dest[editop_list[op][2]]) - 1
      #print('trans', x, y)
      transpose_table[x][y] = transpose_table[x][y] + 1

    elif (editop_list[op][0] == 'replace'): 
      x = get_alphabet_index(dest[editop_list[op][2]]) - 1
      y = get_alphabet_index(src[editop_list[op][1]]) - 1
      #print('replace', x, y)
      substitute_table[x][y] = substitute_table[x][y] + 1


### Creating a model for misspelled words form the file 'misspelled.txt' in a dictionery format

In [49]:
import collections

In [48]:
def read_file_lines(file_name):

  uploaded = files.upload()

  with open(file_name, 'r') as file:
    corpus = file.readlines()
  
  return corpus 


def create_misspellings_model():
  misspellings = read_file_lines('misspellings.txt')
  # misspellings
  
  misspellings_model = collections.defaultdict(list)

  for line in misspellings: 
    line = line.rstrip("\n")
    correct = line.split(", ")
    errors = correct[1].split("\t")

    for word in list(errors):
      misspellings_model[correct[0]].append(word)

  return misspellings_model

### Populating the mapping table based on the misspelled model

In [81]:
def create_mappings_tables():
  misspellings_model = create_misspellings_model()

  for correct, wrong_list in misspellings_model.items():
    
    for wrong in wrong_list: 
      # print(correct, wrong)
      populate_mappings_lists(correct, wrong)
    
    return misspellings_model

In [82]:
misspellings_model_global = create_mappings_tables()

Saving misspellings.txt to misspellings (5).txt


### Generating all the candidate words with respect to the user input

In [83]:
def get_all_candidate_words(user_input):
  tokens = nltk.word_tokenize(corpus)
  candidates = []
  for word in tokens: 
    if (distance(word, user_input) == 1 ):
      candidates.append(word)
    elif (distance(word, user_input) == 0 ):
      return None  
      
  return candidates

In [67]:
def sum_of(arr):
  return np.sum(arr)

### Gives the count of specified characters from the misspellings.txt model.

In [85]:
def count_in_training_data(deleted_chars):
  #misspellings_model = create_misspellings_model()
  
  count = 0
  for wrongs in misspellings_model_global.values(): 
    for word in wrongs:
      count += word.count(deleted_chars) 
      
  return count 

In [86]:
def prior_probability(word):
  tokens = nltk.word_tokenize(corpus)
  return tokens.count(word) / len(tokens)  

### After identifying the operation i.e insert, delete, transpose or substitute, each case probability is calculated 

In [87]:
def probability_of(wrong, right):
  wrong = wrong.lower()
  right = right.lower()
  prob = 0.0 
  editop_list = editops(wrong, right)

  for op in range(len(editop_list)):

    if (editop_list[op][0] == 'insert'):
      x = get_alphabet_index(wrong[editop_list[op][1]-1]) - 1
      y = get_alphabet_index(right[editop_list[op][2]]) - 1
      prob = prob + insert_table[x][y] / sum_of(insert_table[x])

    elif (editop_list[op][0] == 'delete'):
      x = get_alphabet_index(wrong[editop_list[op][1]-1]) - 1
      y = get_alphabet_index(wrong[editop_list[op][2]]) - 1
      deleted_chars = wrong[editop_list[op][1]-1] + wrong[editop_list[op][1]]
      prob = prob + (delete_table[x][y] / delete_table[x][y] + count_in_training_data(deleted_chars))

    elif (editop_list[op][0] == 'replace'): #substitution
      x = get_alphabet_index(right[editop_list[op][2]]) - 1
      y = get_alphabet_index(wrong[editop_list[op][1]]) - 1
      prob = prob + substitute_table[x][y] / sum_of(substitute_table[y])

    elif (op < len(editop_list)-2 and editop_list[op][0] == 'replace' and editop_list[op+1][0] == 'replace'): 
      x = get_alphabet_index(wrong[editop_list[op][1]]) - 1
      y = get_alphabet_index(right[editop_list[op][2]]) - 1
      trans_chars = wrong[editop_list[op][1]-1] + right[editop_list[op][2]]
      prob = prob + (transpose_table[x][y] / transpose_table[x][y] + count_in_training_data(trans_chars))

  return prob 

### For the user specified word (wrong) against each candidate words, error probabilities are calculated multiplied by the prior probabilities of each candidate word. These are then stored in a dictionary and returned.

In [88]:
def calculate_probabilities(user_input, candidates):
  candidate_probabilities = collections.defaultdict(lambda: 0.01)
  for candidate in candidates: 
    # p = probability_of(user_input, candidate)
    # pr = prior_probability(candidate)
    # print('P(x|w).P(w) ', p, pr)
    candidate_probabilities[candidate] = probability_of(user_input, candidate) * prior_probability(candidate)
  
  #print(candidate_probabilities)

  return candidate_probabilities

### Out of all the probabilities, the highest probability is returned.

In [89]:
def argmax(candidate_probabilities): 
  max_val = max(candidate_probabilities.values())
  for key in candidate_probabilities: 
    if (candidate_probabilities[key] == max_val):
      return key


Input following its output

In [99]:
user_input = input("Please enter a word:")
print('')
print('')

candidates= get_all_candidate_words(user_input)
print('')
print('')

if(len(candidates) == 0):
  print ("The word you typed is correct.")
else:
  print('The candidate words are:')
  print(candidates)

  print('')
  print('')

  candidate_probabilities = calculate_probabilities(user_input, candidates)
  print('The probabilities of candidate words are:')
  print(candidate_probabilities)

  print('')
  print('')
  print('Corrected word is: ' , argmax(candidate_probabilities))

Please enter a word:rbha




The candidate words are:
['raha', 'raha', 'raha', 'raha', 'raha', 'raha', 'raha', 'rha', 'rha', 'riha', 'raha', 'riha', 'raha', 'raha', 'raha', 'raha', 'raha', 'raha', 'raha', 'raha', 'rha', 'raha', 'raha', 'raha', 'raha', 'rha', 'rha', 'raha', 'rha', 'rha', 'rha', 'raha']


The probabilities of candidate words are:
defaultdict(<function calculate_probabilities.<locals>.<lambda> at 0x7f04e80f2320>, {'raha': 0.00032957179335299134, 'rha': 0.7921843877454235, 'riha': 1.2910055467633807e-05})


Corrected word is:  rha
